<h1> This notebook translates SQuAD 2.0 to Bangla then corrects and aligns answer spans. This notebook needs translation of SQuaD 1.1 done before to copy already translated contexts</h1>

*   In the translation section select train.json or dev.json to translate train or dev split.

*   Select low and high values to translate a part of dataset if low=0 and high=40 it means the first 40 paragraphs will be translated. The translated dictionary will be saved as "squad2_translated_"+str(low)+"_"+str(high)+"_dict.json"

*   In the merge translation section marge the translated dataset parts giving the correct filename and running the subsection merge parts as many times as the number of parts generated in the previous section. The full dataset is saved as "squad2_translated_temp.json"

*   Finally run the answer correction and alignment section.




In [ ]:
!pip install transformers=="4.25.1" sentencepiece=="0.1.97" fasttext=="0.9.2" utoken=="0.1.8" nltk=="3.8.1" torch=="1.13.1+cu116" numpy=="1.21.6" tqdm=="4.64.1" --quiet

In [ ]:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import numpy as np
from tqdm.notebook import tqdm, trange
import json
import nltk

In [ ]:
!mkdir squad2_data
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O ./squad2_data/train.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O ./squad2_data/dev.json

In [ ]:
f2 = open("./squad2_data/train.json")
data2 = json.load(f2)
f2.close()

In [ ]:
f1 = open('./squad1_data/squad1_translated_merged_temp.json','r')
data1 = json.load(f1)
f1.close()

In [ ]:
save_loc = './squad2_data'

In [ ]:
data2['data'][0]['paragraphs'][0]['context']

# **Translation**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

2022-12-30 02:52:55.537591: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-30 02:52:55.653964: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-30 02:52:55.671285: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-30 02:52:55.991475: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "pretrained_models/nllb-200-3.3B", src_lang="eng_Latn", tgt_lang="ben_Beng"
)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("pretrained_models/nllb-200-3.3B")

In [ ]:
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="eng_Latn", tgt_lang="ben_Beng", device="cuda:0")

In [ ]:
# nltk.download('punkt')
def get_c_sent_list(c_src):
  c_src_sent_list = nltk.sent_tokenize(c_src)
  # print(a_src)
  c_tran_sent_list = []

  for sent in c_src_sent_list:
    c_tran_sent = translator(sent, max_length=400)[0]['translation_text']
    c_tran_sent_list.append(c_tran_sent)

  return c_tran_sent_list, c_src_sent_list

In [ ]:
# Test Run
output,_ = get_c_sent_list(data2['data'][0]['paragraphs'][0]['context'])
print(output)

['নরম্যান (Norman: Nourmands; French: Normands; Latin: Normanni) ছিল একদল মানুষ যারা দশম ও একাদশ শতকে ফ্রান্সের একটি অঞ্চল নরম্যান্ডিতে তাদের নাম দিয়েছিল।', 'তারা ডেনমার্ক, আইসল্যান্ড এবং নরওয়ের নর্স ("নর্মান" "নর্সেম্যান" থেকে এসেছে) আক্রমণকারী এবং জলদস্যুদের বংশধর ছিলেন যারা তাদের নেতা রোলোর অধীনে পশ্চিম ফ্রাঙ্কার রাজা তৃতীয় চার্লসের প্রতি আনুগত্যের শপথ নিতে সম্মত হন।', 'বহু প্রজন্ম ধরে স্থানীয় ফ্রাঙ্ক এবং রোমান-গল জনগোষ্ঠীর সাথে মিশ্রিত হওয়ার ফলে তাদের বংশধররা ক্রমশ পশ্চিম ফ্রাঙ্কার ক্যারোলিংজিয়ান-ভিত্তিক সংস্কৃতির সাথে মিশ্রিত হবে।', 'নরম্যানদের স্বতন্ত্র সাংস্কৃতিক ও জাতিগত পরিচয় প্রথম দিকে দশম শতাব্দীর প্রথমার্ধে আবির্ভূত হয়েছিল এবং পরবর্তী শতাব্দীতে এটি বিকশিত হতে থাকে।']


In [ ]:
import nltk
nltk.download('punkt')
import re
def word_tokenize(text):
    tokens = nltk.word_tokenize(text)
    new_tokens = []
    for token in tokens:
        if token[-1] == "।" and len(token)>1:
            token_1 = token[:-1]
            token_2 = token[-1]
            new_tokens.append(token_1)
            new_tokens.append(token_2)
        elif ("-" in token or  "–" in token) and len(token)>1:
            temp_tokens = token.replace("-", " - ").replace("–", " – ").replace("—"," — ").split(" ")
            new_tokens.extend(temp_tokens)
        else:
            new_tokens.append(token)
    return new_tokens

def detokenize(words):
    """
    Untokenizing a text undoes the tokenizing operation, restoring
    punctuation and spaces to the places that people expect them to be.
    Ideally, `untokenize(tokenize(text))` should be identical to `text`,
    except for line breaks.
    """
    text = ' '.join(words)
    step1 = text.replace("`` ", '"').replace(" ''", '"').replace('‘ ', '‘').replace(' ’', '’').replace('. . .',  '...')
    step2 = step1.replace("( ", "(").replace(" )", ")").replace(" - ","-").replace(" – ","–").replace(" — ","—")
    step3 = re.sub(r' ([.,:;?!%]+)([ \'"`])', r"\1\2", step2)
    step4 = re.sub(r' ([.,:;?!%]+)$', r"\1", step3)
    step5 = step4.replace(" ' ", "'").replace(" & ", "&")
    step6 = step5.replace(" ` ", " '").replace("“ ", "“").replace(" ”", "”")
    step7 = step6.replace(" ।", "।")
    step8 = step7.replace(" %","%").replace("[ ", "[").replace(" ]", "]").replace("{ ", "{").replace(" }", "}")
    return step8.strip()


[nltk_data] Downloading package punkt to /home/dlpc01/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Translating any left-over english digits
digits_en = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
digits_bn = ['০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯']
def translate_digits(s):
  s = list(s)
  for i in range(len(s)):
    if s[i] in digits_en:
      s[i] = digits_bn[digits_en.index(s[i])]
  s = "".join(s)
  return s

In [ ]:
def get_index_c_tran_with_ans(c_src_tokens, c_tran_tokens, a_ch_start_pos_src):
  ch_len = 0
  for ind in range(len(c_src_tokens)):
    ch_len += len(c_src_tokens[ind])
    ch_len += 1 # For space after a sentence
    if(ch_len > a_ch_start_pos_src):
      return ind
  return ind

In [ ]:
# a_src = data['data'][0]['paragraphs'][3]['qas'][1]['answers'][0]['text']
import string
import re
punc = '''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~—।–'''
def get_a_tran(c_tran_with_ans, a_src):
  a_tran = a_src
  if a_src in c_tran_with_ans:
    a_tran = a_src
  elif len(word_tokenize(a_src))==1:
    clean_ans_text = a_src.translate(str.maketrans('', '', punc))
    if re.match('^[0-9]*$', clean_ans_text):
      a_tran = translate_digits(a_src)
  else:
    a_tran = translator(a_src, max_length=400)[0]['translation_text']
    a_tran = translate_digits(a_tran)
  return a_tran

# print(c_tran_token)
# print(a_tran_token)

# **Get Bangla Context from SQUAD 1.1 Translation or Translate**

In [ ]:
for di in tqdm(range(len(data2['data'])),desc="Paragraphs"):
    d = data2['data'][di]
    for p in d['paragraphs']:
        flag = False
        for di1 in range(len(data1['data'])):
            for pi1 in range(len(data1['data'][di1]['paragraphs'])):
                if data1['data'][di1]['paragraphs'][pi1]['context'] == p['context']:
                    flag = True
                    break
            if flag:
                break
        if flag:
          p['bangla_context'] = data1['data'][di1]['paragraphs'][pi1]['bangla_context']
        else:
          c_tran_sent_list, c_src_sent_list = get_c_sent_list(p['context'])
          p['bangla_context'] = c_tran_sent_list

Paragraphs:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
low = 0
high = 40
save_filename = "squad2_translated_only_context_"+str(low)+"_"+str(high)+"_dict"
save_loc = './squad2_data'

In [ ]:
out_file = open(save_loc+'/'+save_filename+'.json', "w")
json.dump(data2, out_file, indent = 4) # save whole data replace parts later
out_file.close()

# **Translate All Questions**

In [ ]:
q_src_all = []
for di in tqdm(range(0,len(data2['data'])),desc="Paragraphs"):
  d = data2['data'][di]
  for p in d['paragraphs']:
    for qas in p['qas']:
        q_src_all.append(qas['question'])


Paragraphs:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
# len(q_src_all)

In [ ]:
import torch
class QDataset(torch.utils.data.Dataset):
    def __init__(self, text):
        self.text = text

    def __getitem__(self, idx):
        return self.text[idx]

    def __len__(self):
        return len(self.text)

    
q_dataset = QDataset(q_src_all)

In [ ]:
q_tran_all = []
for out in tqdm(translator(q_dataset, batch_size=16), total=len(q_dataset)):
    q_tran_all.append(out)

  0%|          | 0/11873 [00:00<?, ?it/s]

In [ ]:
q_tran_all = [q[0]['translation_text'] for q in q_tran_all]

In [ ]:
i=0
for di in tqdm(range(0,len(data2['data'])),desc="Paragraphs"):
  d = data2['data'][di]
  for p in d['paragraphs']:
    for qas in p['qas']:
      qas['q_tran'] = q_tran_all[i]
      i+=1


Paragraphs:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
# for di in tqdm(range(0,1),desc="Paragraphs"):
#   d = data2['data'][di]
#   for p in tqdm(d['paragraphs'],desc="Contexts"):
#     print(p)

In [ ]:
# low = 0
# high = len(data2['data'])
# save_filename = "SQuAD2_translated_cq_dev_"+str(low)+"_"+str(high)+"_dict"
# save_loc = './squad2_data'

In [ ]:
# out_file = open(save_loc+'/'+save_filename+'.json', "w")
# json.dump(data2, out_file, indent = 4) # save whole data replace parts later
# out_file.close()

# **Translate All Answers**

In [ ]:
a_src_all = []
for di in tqdm(range(0,len(data2['data'])),desc="Paragraphs"):
  d = data2['data'][di]
  for p in d['paragraphs']:
    for qas in p['qas']:
        for ans in qas['answers']:
          a_src_all.append(ans['text'])

Paragraphs:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
import torch
class ADataset(torch.utils.data.Dataset):
    def __init__(self, text):
        self.text = text

    def __getitem__(self, idx):
        return self.text[idx]

    def __len__(self):
        return len(self.text)

    
a_dataset = ADataset(a_src_all)

In [ ]:
a_tran_all = []
for out in tqdm(translator(a_dataset, batch_size=16), total=len(a_dataset)):
    a_tran_all.append(out)

  0%|          | 0/20302 [00:00<?, ?it/s]

In [ ]:
a_tran_all = [a[0]['translation_text'] for a in a_tran_all]

In [ ]:
i=0
for di in tqdm(range(len(data2['data'])),desc="Paragraphs"):
  d = data2['data'][di]
  for p in d['paragraphs']:
    for qas in p['qas']:
      for ans in qas['answers']:
        ans['a_tran_temp'] = a_tran_all[i]
        i+=1

Paragraphs:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
# for di in tqdm(range(len(data['data'])),desc="Paragraphs"):
import string
import re
punc = '''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~—।–'''
def get_a_tran(c_tran_with_ans, a_src, a_tran_temp):
  a_tran = a_src
  if a_src in c_tran_with_ans:
    a_tran = a_src
  elif len(word_tokenize(a_src))==1:
    clean_ans_text = a_src.translate(str.maketrans('', '', punc))
    if re.match('^[0-9]*$', clean_ans_text):
      a_tran = translate_digits(a_src)
  else:
    a_tran = a_tran_temp
    a_tran = translate_digits(a_tran)
  return a_tran


for di in tqdm(range(len(data2['data'])),desc="Paragraphs"):
  d = data2['data'][di]
  for p in d['paragraphs']:
    # for qas in tqdm(p['qas'],desc="QuesAns"):
    c_tran_sent_list = p['bangla_context']
    c_src_sent_list = nltk.sent_tokenize(p['context'])
    assert len(c_src_sent_list)==len(c_tran_sent_list)

    for qas in p['qas']:
      q_tran = qas['q_tran']
      for ans in qas['answers']:
        a_ch_start_pos_src = ans['answer_start']
        a_src = ans['text']
        a_tran_temp = ans['a_tran_temp']
        index_c_tran_with_ans = get_index_c_tran_with_ans(c_src_sent_list, c_tran_sent_list, a_ch_start_pos_src)
        a_tran = get_a_tran(c_tran_sent_list[index_c_tran_with_ans], a_src,  a_tran_temp)

        ans['a_tran'] = a_tran
        ans['index_c_tran_with_ans'] = index_c_tran_with_ans

Paragraphs:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
# out_file = open(save_loc+'/'+'squad2_dev_temp_translated_cqa_tran'+'.json', "w")
# json.dump(data2, out_file, indent = 4) # save whole data replace parts later
# out_file.close()

# **Translate All Plausible Answers**

In [ ]:
pa_src_all = []
for di in tqdm(range(0,len(data2['data'])),desc="Paragraphs"):
  d = data2['data'][di]
  for p in d['paragraphs']:
    for qas in p['qas']:
        if 'plausible_answers' in qas:
          for pans in qas['plausible_answers']:
              pa_src = re.sub(r'(?<=\d)[,\.](?=\d)','',pans['text'])
              pa_src_all.append(pa_src)

Paragraphs:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
pa_dataset = ADataset(pa_src_all)

In [ ]:
pa_tran_all = []
for out in tqdm(translator(pa_dataset, batch_size=16), total=len(pa_dataset)):
    pa_tran_all.append(out)

  0%|          | 0/5930 [00:00<?, ?it/s]

In [ ]:
pa_tran_all = [pa[0]['translation_text'] for pa in pa_tran_all]

In [ ]:
i=0
for di in tqdm(range(len(data2['data'])),desc="Paragraphs"):
  d = data2['data'][di]
  for p in d['paragraphs']:
      for qas in p['qas']:
        if 'plausible_answers' in qas:
          for pans in qas['plausible_answers']:
            pans['a_tran_temp'] = pa_tran_all[i]
            i+=1

Paragraphs:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
# for di in tqdm(range(len(data['data'])),desc="Paragraphs"):
for di in tqdm(range(len(data2['data'])),desc="Paragraphs"):
  d = data2['data'][di]
  for p in d['paragraphs']:
    c_tran_sent_list = p['bangla_context']
    c_src_sent_list = nltk.sent_tokenize(p['context'])
    assert len(c_src_sent_list)==len(c_tran_sent_list)
    for qas in p['qas']:
        if 'plausible_answers' in qas:
          for pans in qas['plausible_answers']:
              a_ch_start_pos_src = pans['answer_start']
              a_src = pans['text']
              a_tran_temp = pans['a_tran_temp']
              index_c_tran_with_ans = get_index_c_tran_with_ans(c_src_sent_list, c_tran_sent_list, a_ch_start_pos_src)
              a_tran = get_a_tran(c_tran_sent_list[index_c_tran_with_ans], a_src, a_tran_temp)

              pans['a_tran'] = a_tran
              pans['index_c_tran_with_ans'] = index_c_tran_with_ans

Paragraphs:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
out_file = open(save_loc+'/'+'squad2_temp_translated'+'.json', "w")
json.dump(data2, out_file, indent = 4) # save whole data replace parts later
out_file.close()

In [ ]:
for di in tqdm(range(len(data2['data'])),desc="Paragraphs"):
  d = data2['data'][di]
  for p in d['paragraphs']:
    print(p)
    break;
  break;


Paragraphs:   0%|          | 0/442 [00:00<?, ?it/s]

{'qas': [{'question': 'When did Beyonce start becoming popular?', 'id': '56be85543aeaaa14008c9063', 'answers': [{'text': 'in the late 1990s', 'answer_start': 269, 'a_tran_temp': '১৯৯০ এর দশকের শেষের দিকে', 'a_tran': '১৯৯০ এর দশকের শেষের দিকে', 'index_c_tran_with_ans': 1}], 'is_impossible': False, 'q_tran': 'বিয়োনস কবে থেকে জনপ্রিয় হতে শুরু করে?'}, {'question': 'What areas did Beyonce compete in when she was growing up?', 'id': '56be85543aeaaa14008c9065', 'answers': [{'text': 'singing and dancing', 'answer_start': 207, 'a_tran_temp': 'গান আর নাচ', 'a_tran': 'গান আর নাচ', 'index_c_tran_with_ans': 1}], 'is_impossible': False, 'q_tran': 'বিয়োনস যখন বড় হচ্ছিল তখন কোন কোন ক্ষেত্রে প্রতিযোগিতা করতো?'}, {'question': "When did Beyonce leave Destiny's Child and become a solo singer?", 'id': '56be85543aeaaa14008c9066', 'answers': [{'text': '2003', 'answer_start': 526, 'a_tran_temp': '২০০৩', 'a_tran': '2003', 'index_c_tran_with_ans': 3}], 'is_impossible': False, 'q_tran': 'বেয়নস কখন ডেসটিনিস 

# **Answer span correction and alignment**

In [ ]:
!wget "https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.bn.300.bin.gz"
!gunzip './cc.bn.300.bin.gz'

In [ ]:
import io
# from gensim.models import KeyedVectors
import fasttext
fasttext_model = fasttext.load_model("./cc.bn.300.bin")

In [ ]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

2022-12-30 14:17:31.587949: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-30 14:17:31.677784: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-30 14:17:31.693878: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-30 14:17:31.960937: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [ ]:
bn_to_en_tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5_nmt_bn_en")
bn_to_en_model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5_nmt_bn_en")

In [ ]:
bn_to_en_translator = pipeline('translation', model=bn_to_en_model, tokenizer=bn_to_en_tokenizer, device="cuda:0")

In [ ]:
import string
punc = '''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~—।–'''
uw = 0.001
punc_embed = {}
for p in punc:
  punc_embed[p] = np.random.uniform(-uw,uw,300)

sp_punc1 = np.random.uniform(-uw,uw,300)
sp_punc2 = np.random.uniform(-uw,uw,300)

def get_embedding(word):
    word = word.strip("@")
    # if word=="``" or word=="''" or word=="‘" or word=="’":
    #   word = '"'
    if word=="...." or word=="..." or word=="..":
        return sp_punc1
    if word=="()":
        return sp_punc2
    if len(word)==1 and word in punc:
        return punc_embed[word]
    word = word.translate(str.maketrans('', '', punc))
    # if len(word)>3 and word not in ['ইউনিভার্সিটি','দুটি','সিটি'] and re.search(r'টি$', word):
    #     word = word[:-2]
    # word = re.sub("([০-৯]+)([অ-য়]+)",r"\1", word)
    word_vec = fasttext_model.get_word_vector(word)
    return word_vec

In [ ]:
from utoken import utokenize
from utoken import detokenize
tok = utokenize.Tokenizer(lang_code='ben')
detok = detokenize.Detokenizer(lang_code='ben')

# print(tok.utokenize_string("Dont worry!"))


In [ ]:
import json

f = open('./squad2_data/squad2_temp_translated.json', 'r')
data = json.load(f)
f.close()

In [ ]:
import re
regex_year = re.compile("([০-৯]+)-([০-৯]+)")
digits_dict = {"0":'০', "1": '১', "2": '২', "3": '৩', "4": '৪', "5": '৫', "6": '৬', "7": '৭', "8": '৮', "9": '৯'}
def digit_to_bn(digit):
    return digits_dict[digit.group()]
for di in tqdm(range(len(data['data'])),desc="Paragraphs"):
    d = data['data'][di]
    for p in d['paragraphs']:
        new_c_tran = []
        for c_tran in p['bangla_context']:
            c_tran = re.sub("[0-9]",digit_to_bn, c_tran)
            c_tran = re.sub("([০-৯]+)(th)",r"\1তম", c_tran)
            c_tran = re.sub("([০-৯]+)([অ-য়]+)",r"\1 \2", c_tran)
            c_tran = re.sub("([১-২][০-৯][০-৯][০-৯])([১-২]?[০-৯]?[০-৯][০-৯])",r"\1-\2", c_tran)
            new_c_tran.append(c_tran)
        p['bangla_context'] = new_c_tran

        new_c_tran = []
        # new_c_tran.append(p['bangla_context'][0])
        j=0
        for i,c_tran in enumerate(p['bangla_context']):
            if i!=0 and (new_c_tran[-1][-1]=='.' or len(c_tran)<30):
                new_c_tran[-1]+=(" "+c_tran)
                for qas in p['qas']:
                    for ans in qas['answers']:
                        if ans['index_c_tran_with_ans']>=(i-j):
                            ans['index_c_tran_with_ans']-=1
                j+=1
            else:
                new_c_tran.append(c_tran)                
        p['bangla_context'] = new_c_tran

        for qas in p['qas']:
            qas['q_tran'] = re.sub("[0-9]",digit_to_bn, qas['q_tran'])
            qas['q_tran'] = re.sub("([০-৯]+)(th)",r"\1তম", qas['q_tran'])
            qas['q_tran'] = re.sub("([০-৯]+)([অ-য়]+)",r"\1 \2", qas['q_tran'])
            qas['q_tran'] = re.sub("([১-২][০-৯][০-৯][০-৯])([১-২]?[০-৯]?[০-৯][০-৯])",r"\1-\2", qas['q_tran'])
            for ans in qas['answers']:
                ans['a_tran'] = re.sub("[0-9]",digit_to_bn, ans['a_tran'])
                ans['a_tran'] = re.sub("([০-৯]+)(th)",r"\1তম", ans['a_tran'])
                ans['a_tran'] = re.sub("([০-৯]+)([অ-য়]+)",r"\1 \2", ans['a_tran'])
                ans['a_tran'] = re.sub("([১-২][০-৯][০-৯][০-৯])([১-২]?[০-৯]?[০-৯][০-৯])",r"\1-\2", ans['a_tran'])


Paragraphs:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
digits_en = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]
digits_bn = ['০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯']
for di in tqdm(range(len(data['data'])),desc="Paragraphs"):
    d = data['data'][di]
    for p in d['paragraphs']:
        c_tran_sent_list = p['bangla_context']
        c_tran_final = ""
        for c_tran_sent in c_tran_sent_list:
            c_tran_final += (c_tran_sent + " ")
        if any(n in c_tran_final for n in digits_en):
            print('c_tran: ', c_tran_final)
        for qas in p['qas']:
            if any(n in qas['q_tran'] for n in digits_en):
                print('q_tran: ', qas['q_tran'])
            for ans in qas['answers']:
                if any(n in ans['a_tran'] for n in digits_en):
                    print('a_tran: ', ans['a_tran'])

Paragraphs:   0%|          | 0/35 [00:00<?, ?it/s]

In [ ]:
def get_answer_start(index_c_tran_with_ans, a_tran, c_tran, debug=False):
  c_tran_with_ans = c_tran[index_c_tran_with_ans]
  c_tran_with_ans_token = tok.utokenize_string(c_tran_with_ans).split(" ")
#   a_tran = a_tran.translate(str.maketrans('', '', '''`'",।|'''))
  a_tran_token = tok.utokenize_string(a_tran).split(" ")
#   a_tran_token = [token for token in a_tran_token if token!='']
  if debug:
      print(c_tran_with_ans_token)  
      print(a_tran_token)
  if len(a_tran_token)==0:
    return a_tran, 0, 0 
#   if debug:
#     print(c_tran_with_ans_token)
#     print(a_tran_token)
  ans_vec = np.array([np.array(get_embedding(a_word)) for a_word in a_tran_token])
  #print(ans_vec)
  ans_norm = np.apply_along_axis(np.linalg.norm,1,ans_vec)
  # print(ans_norm)
  lengths = [-1, 0, 1]
  sim_lists = []
  for length in lengths:
    if len(a_tran_token)+length>=len(c_tran_with_ans_token):
        sim_lists.append(np.array([-1]))
        continue
    sim_mat = []
    for i in range(len(a_tran_token)+length):
        c_word = c_tran_with_ans_token[i]
        cw_vec = np.array(get_embedding(c_word))
        cw_norm = np.linalg.norm(cw_vec)
        sim_measure = (np.dot(ans_vec,cw_vec)/ans_norm)/cw_norm
        sim_mat.append(sim_measure)

    sim_mat = np.array(sim_mat)
    # print(sim_mat.shape)
    sim_list = []

    for i in range(len(a_tran_token)+length,len(c_tran_with_ans_token)):
        c_word = c_tran_with_ans_token[i]
        cw_vec = np.array(get_embedding(c_word))
        cw_norm = np.linalg.norm(cw_vec)
        sim_measures = (np.dot(ans_vec,cw_vec)/ans_norm)/cw_norm
        if(sim_mat.shape[0]==0):
            sim_mat = np.array([sim_measures])
        else:
            sim_mat = np.append(sim_mat,np.array([sim_measures]),0)
        sim_mat_cal = sim_mat
        
        similarity = 0
        for j in range(len(a_tran_token)):
            max_index = np.unravel_index(sim_mat_cal.argmax(),sim_mat_cal.shape)
            similarity+= sim_mat_cal[max_index]
            sim_mat_cal = np.delete(sim_mat_cal,max_index[0],0)
            sim_mat_cal = np.delete(sim_mat_cal,max_index[1],1)

        sim_list.append(similarity)
        sim_mat = np.delete(sim_mat,0,0)

    sim_list = np.divide(sim_list,len(a_tran_token))
    # if debug:
    #     print(sim_list)
    sim_lists.append(sim_list)




  if(max(sim_lists[2]) - max(sim_lists[0]) >= 0.08 and max(sim_lists[2]) - max(sim_lists[1]) >= 0.03):
    # print(c_tran_with_ans_token[sim_lists[2].argmax()])
    align_score = max(sim_lists[2])
    best_ans_start = sim_lists[2].argmax()
    best_ans_length = len(a_tran_token)+2#lengths[2]
    
    if debug:
        print('best in equal+2')
  elif(max(sim_lists[1]) - max(sim_lists[0]) >= 0.06):
    # print(c_tran_with_ans_token[sim_lists[1].argmax()])
    align_score = max(sim_lists[1])
    best_ans_start = sim_lists[1].argmax()
    best_ans_length = len(a_tran_token)+1#lengths[1]

    if debug:
        print('best in equal+1')
  else:
    # print(c_tran_with_ans_token[sim_lists[0].argmax()])
    align_score = max(sim_lists[0])
    best_ans_start = sim_lists[0].argmax()
    best_ans_length = len(a_tran_token)#lengths[0]
    
    if debug:
        print('best in equal')

  
  if best_ans_start+best_ans_length>len(c_tran_with_ans_token):
      best_ans_length=len(c_tran_with_ans_token)-best_ans_start+1

  a_tran_final = detok.detokenize_string(" ".join(c_tran_with_ans_token[best_ans_start:best_ans_start+best_ans_length]))
  a_tran_final = re.sub("([০-৯]+)([অ-য়]+)",r"\1 \2", a_tran_final)

  ans_start_ref = c_tran_with_ans.find(a_tran_final)
  if debug:
      print(a_tran_final)  

  ans_start_final = 0
  for i in range(len(c_tran)):
    c_tran_sent = c_tran[i]
    if i == index_c_tran_with_ans:
      ans_start_final+= ans_start_ref  
    if i < index_c_tran_with_ans:        
      ans_start_final += len(c_tran_sent)+1
      
  # print(c_tran_final)
  # print(ans_start_final)
  return a_tran_final, ans_start_final,align_score

In [ ]:
import re
align_punc = '''!"#&'*,./:;<=>?@\^_`-|~—।–'''
align_punc2 = '''!#&'*,/:;<=>?@\^_`-|~—।–'''

regexp = re.compile(r'[A-z]')

In [ ]:
regexp_bn = re.compile(r'[অ-য়]')
def translate_bn_en(text):
    if regexp_bn.search(text):
        return bn_to_en_translator(text)[0]['translation_text']
    else:
        return text

In [ ]:
def get_back_trans_answer_start(c_tran_with_ans_token, a_tran_token, debug=False):
  if len(a_tran_token)==0:
    return a_tran, 0, 0 

  ans_vec = np.array([np.array(get_embedding(a_word)) for a_word in a_tran_token])
  #print(ans_vec)
  ans_norm = np.apply_along_axis(np.linalg.norm,1,ans_vec)
  # print(ans_norm)
  lengths = [-1, 0, 1]
  sim_lists = []
  for length in lengths:
    if len(a_tran_token)+length>=len(c_tran_with_ans_token):
        sim_lists.append(np.array([-1]))
        continue
    sim_mat = []
    for i in range(len(a_tran_token)+length):
        c_word = c_tran_with_ans_token[i]
        cw_vec = np.array(get_embedding(c_word))
        cw_norm = np.linalg.norm(cw_vec)
        sim_measure = (np.dot(ans_vec,cw_vec)/ans_norm)/cw_norm
        sim_mat.append(sim_measure)

    sim_mat = np.array(sim_mat)
    # print(sim_mat.shape)
    sim_list = []

    for i in range(len(a_tran_token)+length,len(c_tran_with_ans_token)):
        c_word = c_tran_with_ans_token[i]
        cw_vec = np.array(get_embedding(c_word))
        cw_norm = np.linalg.norm(cw_vec)
        sim_measures = (np.dot(ans_vec,cw_vec)/ans_norm)/cw_norm
        if(sim_mat.shape[0]==0):
            sim_mat = np.array([sim_measures])
        else:
            sim_mat = np.append(sim_mat,np.array([sim_measures]),0)
        sim_mat_cal = sim_mat
        
        similarity = 0
        for j in range(len(a_tran_token)):
            max_index = np.unravel_index(sim_mat_cal.argmax(),sim_mat_cal.shape)
            similarity+= sim_mat_cal[max_index]
            sim_mat_cal = np.delete(sim_mat_cal,max_index[0],0)
            sim_mat_cal = np.delete(sim_mat_cal,max_index[1],1)

        sim_list.append(similarity)
        sim_mat = np.delete(sim_mat,0,0)

    sim_list = np.divide(sim_list,len(a_tran_token))
    # if debug:
    #     print(sim_list)
    sim_lists.append(sim_list)

  if(max(sim_lists[1]) - max(sim_lists[0]) >= 0.06):
    # print(c_tran_with_ans_token[sim_lists[1].argmax()])
    align_score = max(sim_lists[1])
    best_ans_start = sim_lists[1].argmax()
    best_ans_length = len(a_tran_token)+1#lengths[1]

    if debug:
        print('best in equal+1')
  else:
    # print(c_tran_with_ans_token[sim_lists[0].argmax()])
    align_score = max(sim_lists[0])
    best_ans_start = sim_lists[0].argmax()
    best_ans_length = len(a_tran_token)#lengths[0]
    
    if debug:
        print('best in equal')
  
  if best_ans_start+best_ans_length>len(c_tran_with_ans_token):
      best_ans_length=len(c_tran_with_ans_token)-best_ans_start+1
      
  return best_ans_start ,align_score

In [ ]:
data['data'][0]['paragraphs'][0]

In [ ]:
import copy

tran_data = data
c_tran_tokens = []
c_tran_tokens_len = [len(c_tran_tokens)]
a_tran_tokens = []
a_tran_tokens_len = [len(a_tran_tokens)]

for di in tqdm(range(len(tran_data['data']))):
  d = tran_data['data'][di]
  for p in d['paragraphs']:
    c_tran_sent_list = copy.deepcopy(p['bangla_context'])
    c_tran_final = " ".join(copy.deepcopy(p['bangla_context']))
    p['bangla_context_list'] = c_tran_sent_list
    p['bangla_context'] = c_tran_final
    for qas in p['qas']:
      for ans in qas['answers']:
        a_tran = ans['a_tran']
        index_c_tran_with_ans = ans['index_c_tran_with_ans']
        a_src = ans['text']
        # print('banga question -> ',qas['q_tran'])
        # print('translated bangla answer -> ', a_tran)
        # print('english answer -> ', a_src)
        if len(a_tran)>1 and a_tran[0] in align_punc2:
            a_tran = a_tran[1:]
        if len(a_tran)>1 and a_tran[-1] in align_punc2:
            a_tran = a_tran[:-1]
        a_tran_final, ans_start_final, align_score = get_answer_start(index_c_tran_with_ans, a_tran, c_tran_sent_list,debug=False)
        
        if regexp.search(a_tran) or align_score<0.6: # or  "'" in a_tran_final or (a_tran_final[0] in align_punc or a_tran_final[-1] in align_punc):
            c_tran_with_ans_token = tok.utokenize_string(c_tran_sent_list[index_c_tran_with_ans]).split(" ")
            c_tran_tokens.extend(c_tran_with_ans_token)
            c_tran_tokens_len.extend([len(c_tran_tokens)])

            a_tran_token = tok.utokenize_string(a_tran).split(" ")
            a_tran_tokens.extend(a_tran_token)
            a_tran_tokens_len.extend([len(a_tran_tokens)])

        ans['a_tran_temp'] = a_tran
        ans['a_tran'] = a_tran_final
        ans['a_tran_start'] = ans_start_final
        ans['align_score'] = float(align_score)

  0%|          | 0/35 [00:00<?, ?it/s]

/tmp/ipykernel_42112/1599377179.py:41: RuntimeWarning: invalid value encountered in divide
  sim_measures = (np.dot(ans_vec,cw_vec)/ans_norm)/cw_norm
/tmp/ipykernel_42112/1599377179.py:30: RuntimeWarning: invalid value encountered in divide
  sim_measure = (np.dot(ans_vec,cw_vec)/ans_norm)/cw_norm


In [ ]:
import torch
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, text):
        self.text = text

    def __getitem__(self, idx):
        return self.text[idx].strip("@")

    def __len__(self):
        return len(self.text)


# convert our tokenized data into a torch Dataset
context_dataset = TextDataset(c_tran_tokens)
ans_dataset = TextDataset(a_tran_tokens)

In [ ]:
c_back_tran_tokens = []
for out in tqdm(bn_to_en_translator(context_dataset, batch_size=32), total=len(context_dataset)):
    c_back_tran_tokens.append(out)

  0%|          | 0/200390 [00:00<?, ?it/s]

In [ ]:
ans_back_tran_tokens = []
for out in tqdm(bn_to_en_translator(ans_dataset, batch_size=32), total=len(ans_dataset)):
    ans_back_tran_tokens.append(out)

  0%|          | 0/12835 [00:00<?, ?it/s]

In [ ]:
c_back_tran_tokens = [t[0]['translation_text'] for t in c_back_tran_tokens]

In [ ]:
ans_back_tran_tokens = [t[0]['translation_text'] for t in ans_back_tran_tokens]

In [ ]:
ij=1
for di in tqdm(range(len(tran_data['data']))):
  d = tran_data['data'][di]
  for p in d['paragraphs']:
#     c_tran_sent_list = p['bangla_context']
#     c_tran_final = " ".join(c_tran_sent_list)
#     p['bangla_context_list'] = c_tran_sent_list
#     p['bangla_context'] = c_tran_final
    c_tran_sent_list = p['bangla_context_list']
    for qas in p['qas']:
      for ans in qas['answers']:
        a_tran = ans['a_tran_temp']
        index_c_tran_with_ans = ans['index_c_tran_with_ans']
        a_src = ans['text']
        # print('banga question -> ',qas['q_tran'])
        # print('translated bangla answer -> ', a_tran)
        # print('english answer -> ', a_src)
        if len(a_tran)>1 and a_tran[0] in align_punc2:
            a_tran = a_tran[1:]
        if len(a_tran)>1 and a_tran[-1] in align_punc2:
            a_tran = a_tran[:-1]
        a_tran_final, ans_start_final, align_score = get_answer_start(index_c_tran_with_ans, a_tran, c_tran_sent_list,debug=False)
        
        
        if regexp.search(a_tran) or align_score<0.6: # or  "'" in a_tran_final or (a_tran_final[0] in align_punc or a_tran_final[-1] in align_punc):
            c_tran_with_ans_token = tok.utokenize_string(c_tran_sent_list[index_c_tran_with_ans]).split(" ")
            a_tran_token = tok.utokenize_string(a_tran).split(" ")

            ans_start_final2 ,align_score2 = get_back_trans_answer_start(c_back_tran_tokens[c_tran_tokens_len[ij-1]:c_tran_tokens_len[ij]], ans_back_tran_tokens[a_tran_tokens_len[ij-1]:a_tran_tokens_len[ij]], debug=False)
            ij+=1
            a_tran_final2 = detok.detokenize_string(" ".join(c_tran_with_ans_token[ans_start_final2:ans_start_final2+len(a_tran_token)]))
            a_tran_final2 = re.sub("([০-৯]+)([অ-য়]+)",r"\1 \2", a_tran_final2)        
            ans_start_ref = c_tran_sent_list[index_c_tran_with_ans].find(a_tran_final2)
            ans_start_final2 = 0
            for i in range(len(c_tran_sent_list)):
                c_tran_sent = c_tran_sent_list[i]
                if i == index_c_tran_with_ans:
                    ans_start_final2+= ans_start_ref  
                if i < index_c_tran_with_ans:        
                    ans_start_final2 += len(c_tran_sent)+1

            if align_score2>align_score+0.05:
                align_score = align_score2
                ans_start_final = ans_start_final2
                a_tran_final = a_tran_final2

        ans['a_tran_temp'] = a_tran
        ans['a_tran'] = a_tran_final
        ans['a_tran_start'] = ans_start_final
        ans['align_score'] = float(align_score)

  0%|          | 0/35 [00:00<?, ?it/s]

/tmp/ipykernel_42112/1599377179.py:41: RuntimeWarning: invalid value encountered in divide
  sim_measures = (np.dot(ans_vec,cw_vec)/ans_norm)/cw_norm
/tmp/ipykernel_42112/1599377179.py:30: RuntimeWarning: invalid value encountered in divide
  sim_measure = (np.dot(ans_vec,cw_vec)/ans_norm)/cw_norm
/tmp/ipykernel_42112/2944554396.py:31: RuntimeWarning: invalid value encountered in divide
  sim_measures = (np.dot(ans_vec,cw_vec)/ans_norm)/cw_norm
/tmp/ipykernel_42112/2944554396.py:20: RuntimeWarning: invalid value encountered in divide
  sim_measure = (np.dot(ans_vec,cw_vec)/ans_norm)/cw_norm


In [ ]:
final_out_file = open('./translated_squad/squad2_translated_final_aligned.json', "w")
json.dump(data, final_out_file, indent = 4) # save whole data replace parts later
final_out_file.close()

In [ ]:
for di in range(400,409):
    d = tran_data['data'][di]
    for p in d['paragraphs']:
        #print('english context -> ', p['context'])
        for qas in p['qas']:
            for ans in qas['answers']:
                if ans['align_score']<0.6:
                    print('bangla context with ans -> ', p['bangla_context_list'][ans['index_c_tran_with_ans']])
                    print('banga question -> ',qas['q_tran'])
                    print('align_score -> ', ans['align_score'])
                    print('final bangla answer span -> ', ans['a_tran'])
                    print('english answer -> ', ans['text'])